<a href="https://colab.research.google.com/github/ykitaguchi77/CongenitalGlaucoma_AI_project/blob/main/Hum_AI_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Comparison between AI and human prediction**

In [20]:
import codecs
import pandas as pd
import os
import numpy as np
import random
import shutil
import re
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_curve, auc
import pandas as pd
from PIL import Image
import tempfile
import time
import glob
import copy
import pickle
import itertools
import matplotlib.pyplot as plt
import statistics

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
csv_path = "/content/drive/MyDrive/研究/進行中の研究/先天性緑内障プロジェクト/Hum_pred/result/AI_2022-05-09.csv"

#Open reslut_csv
with codecs.open(csv_path, "r", "Shift-JIS", "ignore") as file:
        df = pd.read_csv(file, index_col=None, header=0)
df

,Unnamed: 0,path,label,AI,Usui,Okazaki,Shimojyo,Fujino,opacity,no opacity,bilateral,unilateral,skin
0,0,F:\先天性緑内障dataset_for_article_orig_2cont_exo834...,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,1,F:\先天性緑内障dataset_for_article_orig_2gla_ortho82...,1,1,1,0,1,0,0.0,1.0,0.0,1.0,0.0
2,2,F:\先天性緑内障dataset_for_article_orig_2cont_exo942...,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,3,F:\先天性緑内障dataset_for_article_orig_2cont_exo270...,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,4,F:\先天性緑内障dataset_for_article_orig_2gla_ortho84...,1,1,1,1,1,1,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,65,F:\先天性緑内障dataset_for_article_orig_2cont_exo749...,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
66,66,F:\先天性緑内障dataset_for_article_orig_2gla_exo3229...,1,1,1,1,1,1,1.0,0.0,0.0,1.0,1.0
67,67,F:\先天性緑内障dataset_for_article_orig_2gla_ortho44...,1,1,1,1,1,1,0.0,1.0,0.0,1.0,1.0
68,68,F:\先天性緑内障dataset_for_article_orig_2gla_eso5393...,1,1,0,0,1,0,1.0,0.0,0.0,1.0,0.0


In [49]:
#Open reslut_csv
with codecs.open(csv_path, "r", "Shift-JIS", "ignore") as file:
        df = pd.read_csv(file, index_col=None, header=0)

df_opacity = df[df["opacity"] == 1]
df_no_opacity = df[df["no opacity"] == 1]
df_bilateral = df[df["bilateral"] == 1]
df_unilateral = df[df["unilateral"] == 1]
df_skin = df[df["skin"] == 1]
df_no_skin = df[df["skin"] == 0]

title = ["opacity", "no_opacity", "bilateral", "unilateral", "skin", "no_skin"]

k=0
for df in [df_opacity, df_no_opacity, df_bilateral, df_unilateral, df_skin, df_no_skin]:

    AI_num = len(df[df["AI"]==1])
    AI = AI_num/len(df)

    Human_num = 0
    for grader in ["Usui", "Okazaki", "Shimojyo", "Fujino"]:
        Human_num += len(df[df[grader]==1])
    Human = Human_num / (len(df)*4)

    print(f"{title[k]}_AI: {AI} ({AI_num}/{len(df)}), {title[k]}_Human: {Human} ({Human_num}/{len(df)*4})" )
    k+=1

opacity_AI: 0.9583333333333334 (23/24), opacity_Human: 0.7916666666666666 (76/96)
no_opacity_AI: 0.7272727272727273 (8/11), no_opacity_Human: 0.3409090909090909 (15/44)
bilateral_AI: 1.0 (12/12), bilateral_Human: 0.6666666666666666 (32/48)
unilateral_AI: 0.8260869565217391 (19/23), unilateral_Human: 0.6413043478260869 (59/92)
skin_AI: 1.0 (4/4), skin_Human: 0.75 (12/16)
no_skin_AI: 0.8709677419354839 (27/31), no_skin_Human: 0.6370967741935484 (79/124)


In [23]:
len(df_opacity[df_opacity['AI']==1])

,Unnamed: 0,path,label,AI,Usui,Okazaki,Shimojyo,Fujino,opacity,no opacity,bilateral,unilateral,skin
4,4,F:\先天性緑内障dataset_for_article_orig_2gla_ortho84...,1,1,1,1,1,1,1.0,0.0,0.0,1.0,0.0
9,9,F:\先天性緑内障dataset_for_article_orig_2gla_ortho88...,1,1,1,1,1,1,1.0,0.0,1.0,0.0,0.0
21,21,F:\先天性緑内障dataset_for_article_orig_2gla_ortho92...,1,1,1,1,1,1,1.0,0.0,0.0,1.0,0.0
22,22,F:\先天性緑内障dataset_for_article_orig_2gla_eso781_...,1,1,1,1,1,1,1.0,0.0,0.0,1.0,0.0
24,24,F:\先天性緑内障dataset_for_article_orig_2gla_ortho22...,1,1,1,1,1,1,1.0,0.0,0.0,1.0,0.0
26,26,F:\先天性緑内障dataset_for_article_orig_2gla_ortho90...,1,1,1,1,1,1,1.0,0.0,1.0,0.0,0.0
28,28,F:\先天性緑内障dataset_for_article_orig_2gla_eso4725...,1,1,1,1,1,1,1.0,0.0,1.0,0.0,0.0
30,30,F:\先天性緑内障dataset_for_article_orig_2gla_ortho71...,1,1,0,0,1,0,1.0,0.0,1.0,0.0,0.0
32,32,F:\先天性緑内障dataset_for_article_orig_2gla_ortho81...,1,1,0,0,1,1,1.0,0.0,0.0,1.0,0.0
33,33,F:\先天性緑内障dataset_for_article_orig_2gla_ortho43...,1,1,1,0,1,1,1.0,0.0,0.0,1.0,0.0


In [48]:
from scipy.stats import fisher_exact
import scipy.stats as st

def chi_square(title, square):
    df = pd.DataFrame(square) 
    x2, p, dof, e = st.chi2_contingency(df,correction=False)
    print(title)
    print(square)
    print(f'p値 　　　= {p :.3f}')
    print(f'カイ2乗値 = {x2:.2f}')
    print(f'自由度　  = {dof}')
    print("")

def fisher_exact(title, square):
    df = pd.DataFrame(square) 
    odds, p = st.fisher_exact(df, alternative='two-sided')
    print("fisher_exact: "+title)
    print(square)
    print(f'odds ratio 　　　= {odds :.3f}')
    print(f'p値 　　　= {p :.3f}')
    print("")

def calc_num(df):
    total_num = len(df)
    AI_detect = len(df[df["AI"]==1]) 
    Human_detect = 0
    for grader in ["Usui", "Okazaki", "Shimojyo", "Fujino"]:
        Human_detect += len(df[df[grader]==1])
    return total_num, AI_detect, Human_detect

opacity, AI_opacity, Human_opacity = calc_num(df_opacity)
no_opacity, AI_no_opacity, Human_no_opacity = calc_num(df_no_opacity)
bilateral, AI_bilateral, Human_bilateral, = calc_num(df_bilateral,)
unilateral, AI_unilateral, Human_unilateral = calc_num(df_unilateral)
skin, AI_skin, Human_skin = calc_num(df_skin)
no_skin, AI_no_skin, Human_no_skin = calc_num(df_no_skin)

num_grader = 4


chi_square("AI vs Human: opacity", [[AI_opacity,opacity-AI_opacity], [Human_opacity,opacity*num_grader-Human_opacity]])
chi_square("AI vs Human: no_opacity", [[AI_no_opacity,no_opacity-AI_no_opacity], [Human_no_opacity,no_opacity*num_grader-Human_no_opacity]])
chi_square("AI vs Human: bilateral", [[AI_bilateral,bilateral-AI_bilateral], [Human_bilateral,bilateral*num_grader-Human_bilateral]])
chi_square("AI vs Human: unilateral", [[AI_unilateral,unilateral-AI_unilateral], [Human_unilateral,unilateral*num_grader-Human_unilateral]])
chi_square("AI vs Human: skin", [[AI_skin,skin-AI_skin], [Human_skin,skin*num_grader-Human_skin]])
chi_square("AI vs Human: no_skin", [[AI_no_skin,no_skin-AI_no_skin], [Human_no_skin,no_skin*num_grader-Human_no_skin]])

chi_square("AI_opacity vs no opaciity", [[AI_opacity,opacity-AI_opacity], [AI_no_opacity,no_opacity-AI_no_opacity]])
chi_square("AI_bilateral vs unilateral", [[AI_bilateral,bilateral-AI_bilateral], [AI_unilateral,unilateral-AI_unilateral]])
chi_square("AI_skin vs no skin", [[AI_skin,skin-AI_skin],[AI_no_skin,no_skin-AI_no_skin]])
chi_square("Hum_opacity vs no opaciity", [[Human_opacity,opacity*num_grader-Human_opacity],[Human_no_opacity,no_opacity*num_grader-Human_no_opacity]])
chi_square("Hum_bilateral vs unilateral", [[Human_bilateral,bilateral*num_grader-Human_bilateral], [Human_unilateral,unilateral*num_grader-Human_unilateral]])
chi_square("Hum_skin vs no skin", [[Human_skin,skin*num_grader-Human_skin], [Human_no_skin,no_skin*num_grader-Human_no_skin]])

print("")

fisher_exact("AI vs Human: opacity", [[AI_opacity,opacity-AI_opacity], [Human_opacity,opacity*num_grader-Human_opacity]])
fisher_exact("AI vs Human: no_opacity", [[AI_no_opacity,no_opacity-AI_no_opacity], [Human_no_opacity,no_opacity*num_grader-Human_no_opacity]])
fisher_exact("AI vs Human: bilateral", [[AI_bilateral,bilateral-AI_bilateral], [Human_bilateral,bilateral*num_grader-Human_bilateral]])
fisher_exact("AI vs Human: unilateral", [[AI_unilateral,unilateral-AI_unilateral], [Human_unilateral,unilateral*num_grader-Human_unilateral]])
fisher_exact("AI vs Human: skin", [[AI_skin,skin-AI_skin], [Human_skin,skin*num_grader-Human_skin]])
fisher_exact("AI vs Human: no_skin", [[AI_no_skin,no_skin-AI_no_skin], [Human_no_skin,no_skin*num_grader-Human_no_skin]])

fisher_exact("AI_opacity vs no opaciity", [[AI_opacity,opacity-AI_opacity], [AI_no_opacity,no_opacity-AI_no_opacity]])
fisher_exact("AI_bilateral vs unilateral", [[AI_bilateral,bilateral-AI_bilateral], [AI_unilateral,unilateral-AI_unilateral]])
fisher_exact("AI_skin vs no skin", [[AI_skin,skin-AI_skin],[AI_no_skin,no_skin-AI_no_skin]])
fisher_exact("Hum_opacity vs no opaciity", [[Human_opacity,opacity*num_grader-Human_opacity],[Human_no_opacity,no_opacity*num_grader-Human_no_opacity]])
fisher_exact("Hum_bilateral vs unilateral", [[Human_bilateral,bilateral*num_grader-Human_bilateral], [Human_unilateral,unilateral*num_grader-Human_unilateral]])
fisher_exact("Hum_skin vs no skin", [[Human_skin,skin*num_grader-Human_skin], [Human_no_skin,no_skin*num_grader-Human_no_skin]])

AI vs Human: opacity
[[23, 1], [76, 20]]
p値 　　　= 0.055
カイ2乗値 = 3.69
自由度　  = 1

AI vs Human: no_opacity
[[8, 3], [15, 29]]
p値 　　　= 0.020
カイ2乗値 = 5.40
自由度　  = 1

AI vs Human: bilateral
[[12, 0], [32, 16]]
p値 　　　= 0.020
カイ2乗値 = 5.45
自由度　  = 1

AI vs Human: unilateral
[[19, 4], [59, 33]]
p値 　　　= 0.090
カイ2乗値 = 2.88
自由度　  = 1

AI vs Human: skin
[[4, 0], [12, 4]]
p値 　　　= 0.264
カイ2乗値 = 1.25
自由度　  = 1

AI vs Human: no_skin
[[27, 4], [79, 45]]
p値 　　　= 0.012
カイ2乗値 = 6.27
自由度　  = 1

AI_opacity vs no opaciity
[[23, 1], [8, 3]]
p値 　　　= 0.046
カイ2乗値 = 3.98
自由度　  = 1

AI_bilateral vs unilateral
[[12, 0], [19, 4]]
p値 　　　= 0.125
カイ2乗値 = 2.36
自由度　  = 1

AI_skin vs no skin
[[4, 0], [27, 4]]
p値 　　　= 0.445
カイ2乗値 = 0.58
自由度　  = 1

Hum_opacity vs no opaciity
[[76, 20], [15, 29]]
p値 　　　= 0.000
カイ2乗値 = 26.95
自由度　  = 1

Hum_bilateral vs unilateral
[[32, 16], [59, 33]]
p値 　　　= 0.765
カイ2乗値 = 0.09
自由度　  = 1

Hum_skin vs no skin
[[12, 4], [79, 45]]
p値 　　　= 0.373
カイ2乗値 = 0.79
自由度　  = 1


fisher_exact: AI vs Human: opa

In [36]:
str = "[[AI_unilateral,unilateral-AI_unilateral], [Human_unilateral,unilateral*num_grader-Human-unilateral]])"
str.replace("unilateral", "no_skin")

'[[AI_no_skin,no_skin-AI_no_skin], [Human_no_skin,no_skin*num_grader-Human-no_skin]])'

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score

#Open reslut_csv
with codecs.open(csv_path, "r", "Shift-JIS", "ignore") as file:
        df = pd.read_csv(file, index_col=None, header=0)

def specificity_score(label, pred):
    tn, fp, fn, tp = confusion_matrix(label, pred).flatten()
    return tn / (tn + fp)


accuracy_list, precision_list, recall_list, specificity_list, f1_score_list = [], [], [], [], []
for grader in ["AI", "Usui", "Okazaki", "Shimojyo", "Fujino"]:
    Y_test = df["label"]
    Y_pred = df[grader]

    tn, fp, fn, tp = confusion_matrix(Y_test, Y_pred).ravel()
    #print(tp, fn, fp, tn)

    print(f"grader: {grader}")
    #print('confusion matrix = \n', confusion_matrix(Y_test, Y_pred))
    print(f'Accuracy : {accuracy_score(Y_test, Y_pred)}')
    print(f'Precision (true positive rate) : {precision_score(Y_test, Y_pred)}')
    print(f'Recall (sensitivity): {recall_score(Y_test, Y_pred)}')
    print(f'Specificity : {specificity_score(Y_test, Y_pred)}')
    print(f'F1 score : {f1_score(Y_test, Y_pred)}')
    print("")

    accuracy_list.append(accuracy_score(Y_test, Y_pred))
    precision_list.append(precision_score(Y_test, Y_pred))
    recall_list.append(recall_score(Y_test, Y_pred))
    specificity_list.append(specificity_score(Y_test, Y_pred))
    f1_score_list.append(f1_score(Y_test, Y_pred))

print("Human")
print(f"Accuracy: {statistics.mean(accuracy_list)}±{statistics.stdev(accuracy_list)}")
print(f"Precision: {statistics.mean(precision_list)}±{statistics.stdev(precision_list)}")
print(f"Recall: {statistics.mean(recall_list)}±{statistics.stdev(recall_list)}")
print(f"Specificity: {statistics.mean(specificity_list)}±{statistics.stdev(specificity_list)}")
print(f"f1_score: {statistics.mean(f1_score_list)}±{statistics.stdev(f1_score_list)}")

grader: AI
Accuracy : 0.9
Precision (true positive rate) : 0.9117647058823529
Recall (sensitivity): 0.8857142857142857
Specificity : 0.9142857142857143
F1 score : 0.8985507246376812

grader: Usui
Accuracy : 0.8142857142857143
Precision (true positive rate) : 1.0
Recall (sensitivity): 0.6285714285714286
Specificity : 1.0
F1 score : 0.7719298245614035

grader: Okazaki
Accuracy : 0.7428571428571429
Precision (true positive rate) : 1.0
Recall (sensitivity): 0.4857142857142857
Specificity : 1.0
F1 score : 0.6538461538461539

grader: Shimojyo
Accuracy : 0.8857142857142857
Precision (true positive rate) : 0.9090909090909091
Recall (sensitivity): 0.8571428571428571
Specificity : 0.9142857142857143
F1 score : 0.8823529411764706

grader: Fujino
Accuracy : 0.8
Precision (true positive rate) : 0.9565217391304348
Recall (sensitivity): 0.6285714285714286
Specificity : 0.9714285714285714
F1 score : 0.7586206896551724

Human
Accuracy: 0.8285714285714286±0.06468132241526725
Precision: 0.955475470820739